### **JL101 2.4 3/20/2028**

<table style="font-size: small; width: 100%;">
  <tr>
    <td style="vertical-align: top; width: 50%;">
      <table style="font-size: small;">
        <tr><td>銘柄名</td><td>利付国庫債券(20年) 第101回</td></tr>
        <tr><td>ISIN</td><td>JP1201011853</td></tr>
        <tr><td>発行日</td><td>2008年5月30日</td></tr>
        <tr><td>表面利率</td><td>2.4%</td></tr>
        <tr><td>償還予定日</td><td>2028年3月20日</td></tr>
      </table>
    </td>
    <td style="vertical-align: top; width: 50%;">
      <table style="font-size: small;">
        <tr><td>取引日</td><td>2021年7月30日</td></tr>
        <tr><td>受渡日数</td><td>T+1</td></tr>
        <tr><td>単利</td><td>-0.125%</td></tr>
        <tr><td>価格</td><td>116.8810</td></tr>
        <tr><td>複利(s.a.)</td><td>-0.134%</td></tr>
      </table>
    </td>
  </tr>
</table>

- myABBR.pyでは、**dcA365** =  ql.Actual365Fixed()を定義 <small>(8ページ参照)</small>
- 新たに **dcA365n** =  ql.Actual365Fixed(ql.Actual365Fixed.NoLeap)を追加
- dcA365nは分子でも閏年を無視する日数計算法でJGBの利回り計算で必要となる

<div align='center'>
<img src='pic/JL101govtYA.png' width='600'>
</div>

**テキスト113ページ、図4-2と比較**

In [1]:
from myABBR import * ; import myUtil as mu
# 0. 初期値設定
tradeDT,             effDT,         matDT,       cpnRT             =\
jDT(2021,7,30), jDT(2008,5,30), jDT(2028,3,20),   2.4/100

settDT,                           mPRC,                            =\
calJP.advance(tradeDT, Tp1, DD), 116.881 ; setEvDT(tradeDT)

# 1. スケジュールオブジェクトと債券オブジェクトの作成
jgbSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, unADJ,unADJ, dtGENb, EoMf)
jgbOBJ = ql.FixedRateBond(Tp1, parAMT, jgbSCD, [cpnRT], dcA365n)
jgbOBa = ql.FixedRateBond(Tp1, parAMT, jgbSCD, [cpnRT], dcA365)

# 価格 >> 利回り計算
PrToYLD = jgbOBJ.bondYield(cP(mPRC), dcA365n, cmpdCMP, freqSA)
print(f'価格{mPRC: .4f}での複利(欧米式): {PrToYLD: .6%}')

価格 116.8810での複利(欧米式): -0.133885%


**テキスト115ページ、図4-3と比較**

In [2]:
# 2. 利回り >> 価格計算
# jgbSCDのメソッド    
prevCpnDT = jgbSCD.previousDate(settDT)
accruDays = dcA365.dayCount(prevCpnDT, settDT)

# jgbOBJのメソッド
accruAMT = jgbOBa.accruedAmount()
cleanPRC = jgbOBJ.cleanPrice(PrToYLD, dcA365n, cmpdCMP, freqSA)
dirtyPRC = jgbOBJ.dirtyPrice(PrToYLD, dcA365n, cmpdCMP, freqSA)
# 保存
dfPRC = pd.DataFrame([ 
            ['受渡日', settDT.to_date()], ['前回利払日',prevCpnDT.to_date()],
            ['経過日数', accruDays     ], ['経過利息',accruAMT],
            ['---- ---- ----', ''     ],
            ['クリーン価格', cleanPRC  ], ['利含み価格',dirtyPRC], 
            ['---- ---- ----', ''   ] ], columns=['価格計算',''])   ; dfPRC

,価格計算,
0,受渡日,2021-08-02
1,前回利払日,2021-03-20
2,経過日数,135
3,経過利息,0.887671
4,---- ---- ----,
5,クリーン価格,116.881001
6,利含み価格,117.768672
7,---- ---- ----,


**テキスト117ページ、図4-4と比較**

In [3]:
# 3. リスク指標の算出
# InterestRateオブジェクトの作成
iRateOBJ = ql.InterestRate(PrToYLD, dcA365n, cmpdCMP, freqSA)

# BondFunctionによる算出
MacDUR = ql.BondFunctions.duration(       jgbOBJ, iRateOBJ, ql.Duration.Macaulay)
ModDUR = ql.BondFunctions.duration(       jgbOBJ, iRateOBJ, ql.Duration.Modified)
BPV    = ql.BondFunctions.basisPointValue(jgbOBJ, iRateOBJ)
convX  = ql.BondFunctions.convexity(      jgbOBJ, iRateOBJ)
# 保存
dfRSK = pd.DataFrame([ 
            ['マコーレーDur.', MacDUR],  ['修正Dur.', ModDUR],
            ['BPV', BPV],              ['Convexity', convX],
            ['---- ---- ----', ''] ],  columns=['リスク指標',''])   ; dfRSK

,リスク指標,
0,マコーレーDur.,6.168071
1,修正Dur.,6.172203
2,BPV,-0.072689
3,Convexity,43.061248
4,---- ---- ----,


**図4.6 (121ページ、bondCashFlow関数を利用)**
- dcAAbを使い、3つ目のjgbOBcオブジェクトを作成
- キャッシュフロー表は単なる確認画面であり、dcAAbを使っても問題はない
- ただ、Present Valueを計算したセルではdirtyPriceと異なっている点に注意

In [4]:
jgbOBc = ql.FixedRateBond(Tp1, parAMT, jgbSCD, [cpnRT], dcAAb)
dfBND  = mu.bondCashFlow(jgbOBc, ir=iRateOBJ)
display(dfBND.style.format(fmtFUT))
print  (f'dirty price(hc): {(dfBND.amount*dfBND.DF).sum():,.5f}')

,payDate,coupon,accruStart,accruEnd,amount,DF
0,2021-09-21,2.4000%,2021-03-20,2021-09-20,1.2000,1.000183
1,2022-03-22,2.4000%,2021-09-20,2022-03-20,1.2000,1.000852
2,2022-09-20,2.4000%,2022-03-20,2022-09-20,1.2000,1.001520
3,2023-03-20,2.4000%,2022-09-20,2023-03-20,1.2000,1.002186
4,2023-09-20,2.4000%,2023-03-20,2023-09-20,1.2000,1.002862
5,2024-03-21,2.4000%,2023-09-20,2024-03-20,1.2000,1.003532
6,2024-09-20,2.4000%,2024-03-20,2024-09-20,1.2000,1.004207
7,2025-03-21,2.4000%,2024-09-20,2025-03-20,1.2000,1.004877
8,2025-09-22,2.4000%,2025-03-20,2025-09-20,1.2000,1.005560
9,2026-03-23,2.4000%,2025-09-20,2026-03-20,1.2000,1.006231


dirty price(hc): 117.76872


**JGBクラスと単利計算メソッド**  
- JSDAの正確な単利計算式 <small>(残存日数で計算する点に注意)</small>  
$$
\scriptsize
\text{単価} = 
\frac{(36500 + 利率 \times 残存日数) \times 100}
     {36500 + 利回り \times 残存日数}
$$   

In [5]:
##### JGB クラス #####    
class JapanFixedRateBond(ql.FixedRateBond):
    def __init__(self, settDS, faceAMT, bondSCD,cpn, dcBond, 
                 paymentConvention=ql.Following,
                 redemption=100.0, issueDate=ql.Date()      ):
      super().__init__(settDS, faceAMT, bondSCD, cpn, dcBond, 
                 paymentConvention, redemption, issueDate   )

    def SimplePrice(self, sYLD, cpnRT, matDS):
        return 100*(365+cpnRT*matDS)/(365+sYLD*matDS)  # 100円当りの価格

    def SimpleYield(self, clnPR, cpnRT, matDS):
        return  ( 100*(365+ cpnRT*matDS)/clnPR - 365 )/matDS  # 実数

JL101 = JapanFixedRateBond(Tp1, parAMT, jgbSCD, [cpnRT], dcA365n)
sYLD  = JL101.SimpleYield(116.881,     cpnRT, dcA365n.dayCount(settDT, matDT))
sPRC  = JL101.SimplePrice(-0.1250/100, cpnRT, dcA365n.dayCount(settDT, matDT))
print(f'価格->単利: {sYLD:,.5%},  '   f'単利->価格: {sPRC:,.7f}')

価格->単利: -0.12500%,  単利->価格: 116.8810001


In [6]:
JL101 = mu.JapanFixedRateBond(Tp1, parAMT, jgbSCD, [cpnRT], dcA365n)
sYLD  = JL101.SimpleYield(116.881,     cpnRT, dcA365n.dayCount(settDT, matDT))
sPRC  = JL101.SimplePrice(-0.1250/100, cpnRT, dcA365n.dayCount(settDT, matDT))
print(f'価格->単利: {sYLD:,.5%},  '   f'単利->価格: {sPRC:,.7f}')

価格->単利: -0.12500%,  単利->価格: 116.8810001
